<a href="https://colab.research.google.com/github/rhaagsma/Projeto_Final_Marvel/blob/main/ProjetoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fazendo Importações e passando os parâmetros necessários para realizar as resquisições

In [ ]:
import hashlib
import time
import requests
from google.colab import userdata
import sqlite3
import pandas as pd

public_key = userdata.get('public_key')
private_key = userdata.get('private_key')

ts = str(time.time())
to_hash = ts + private_key + public_key
hash_digest = hashlib.md5(to_hash.encode()).hexdigest()
auth_params = {
        "ts": ts,
        "apikey": public_key,
        "hash": hash_digest
    }

In [ ]:
base_url = "https://gateway.marvel.com/v1/public/characters"

response = requests.get(base_url, auth_params)
resposta = response.json()


In [ ]:
c = 0  # Inicializa o offset
personagens = pd.DataFrame()

while c < resposta['data']['total']:
    auth_params['offset'] = c  # Atualiza o offset na requisição
    response = requests.get(base_url, auth_params,)
    response = response.json()

    # Verifica se a resposta contém personagens
    if not response['data']['results']:
        break  # Sai do loop se não houver personagens

    response = pd.json_normalize(response['data']['results'])
    df1 = response
    personagens = pd.concat([personagens, df1])
    c += len(df1)

NameError: name 'resposta' is not defined

In [ ]:
personagens.head()

,id,name,description,modified,resourceURI,urls,thumbnail.path,thumbnail.extension,comics.available,comics.collectionURI,comics.items,comics.returned,series.available,series.collectionURI,series.items,series.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,1011334,3-D Man,,2014-04-29T14:18:17+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/c/e0/53...,jpg,12,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,12,3,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,21,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1
1,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,2013-09-18T15:54:04+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/3/20/52...,jpg,4,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,4,2,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,7,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,7,0,https://gateway.marvel.com/v1/public/character...,[],0
2,1009144,A.I.M.,AIM is a terrorist organization bent on destro...,2013-10-17T14:41:30+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/6/20/52...,jpg,53,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,35,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,61,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
3,1010699,Aaron Stack,,1970-01-01T00:00:00+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,jpg,14,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,14,3,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,28,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
4,1009146,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",2014-06-27T19:39:07+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/9/50/4c...,jpg,55,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,28,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,68,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1


In [ ]:
conn = sqlite3.connect('banco.db')
columns = ['id', 'name','description']

personagens[columns].to_sql('characters', conn, if_exists='replace', index=False)

conn.close()

In [ ]:
characters = pd.read_csv('TodosPersonagens.csv')
pd.set_option('display.max_columns', None)

In [ ]:
comics = pd.read_csv('TodosComics.csv', quotechar='"')
pd.set_option('display.max_columns', None)
comics_dados = comics.drop(columns=['issueNumber','thumbnail.path', 'thumbnail.extension', 'variantDescription', 'isbn', 'diamondCode','upc','issn','format','textObjects','resourceURI',
                              'urls','variants','collections','collectedIssues','images','series.resourceURI','series.name','creators.collectionURI','creators.items','creators.returned',
                              'characters.collectionURI','characters.items','stories.collectionURI','stories.items','events.available','events.collectionURI','events.items','modified','ean',
                              'description','creators.available','characters.available','characters.returned','stories.available','stories.returned','events.returned'])


In [ ]:
# Expressão regular para encontrar um ano de 4 dígitos entre parênteses
comics_dados['year'] = comics_dados['title'].str.extract(r'\((\d{4})\)').fillna(0).astype(int)


In [ ]:
comics_dados['prices'].head(5).tolist()


[[{'type': 'printPrice', 'price': 0}],
 [{'type': 'printPrice', 'price': 0}],
 [{'type': 'printPrice', 'price': 0},
  {'type': 'digitalPurchasePrice', 'price': 0}],
 [{'type': 'printPrice', 'price': 3.99}],
 [{'type': 'printPrice', 'price': 0}]]

In [ ]:
def extrair_preco(lista_precos):
    if isinstance(lista_precos, list):
        for item in lista_precos:
            if item.get('type') == 'printPrice':
                return item.get('price')
    return 0

comics_dados['price'] = comics_dados['prices'].apply(extrair_preco)


In [ ]:
comics_dados.head()

,Unnamed: 0,id,digitalId,title,pageCount,dates,prices,year,price
0,0,82967,0,Marvel Previews (2017),112,"[{'type': 'onsaleDate', 'date': '2099-10-30T00...","[{'type': 'printPrice', 'price': 0}]",2017,0.00
1,1,82965,0,Marvel Previews (2017),152,"[{'type': 'onsaleDate', 'date': '2099-08-28T00...","[{'type': 'printPrice', 'price': 0}]",2017,0.00
2,2,82970,52952,Marvel Previews (2017),112,"[{'type': 'onsaleDate', 'date': '2099-01-29T00...","[{'type': 'printPrice', 'price': 0}, {'type': ...",2017,0.00
3,3,61137,0,The Punisher (2016) #5 (Cosplay Variant),32,"[{'type': 'onsaleDate', 'date': '2029-12-31T00...","[{'type': 'printPrice', 'price': 3.99}]",2016,3.99
4,4,37497,0,MARVELS (1994) #1,0,"[{'type': 'onsaleDate', 'date': '2029-12-31T00...","[{'type': 'printPrice', 'price': 0}]",1994,0.00
